In [1]:
import pymongo

from pprint import pprint
from pymongo import MongoClient
client = MongoClient('192.168.2.70', 27017)
db=client['fruitsandveggies']
collection = db['aggregate-posts-out_3']

In [2]:
import spacy
from spacy import displacy

# TODO test lg vs sm
# TODO test spacy core in binder
#nlp_sm = spacy.load('en_core_web_sm')
nlp_md = spacy.load('en_core_web_md')
#nlp_lg = spacy.load('en_core_web_lg')


In [3]:
nlp = nlp_md

In [4]:
for word in nlp.Defaults.stop_words:
    lex = nlp.vocab[word]
    lex.is_stop = True

In [41]:
collection = db['aggregate-posts-out_2']
posts = collection.find({})
# Note: if pulling the data from a database, post['thread_title'] needs to be used

post_tuples = []
for post in posts:
   post_tuples.append((post['_id'],post['user'],nlp(post['content'])))

post_tuples.append((_,_,nlp("i think that i'm retarded")))

# for post in posts:
#     content_doc = nlp(post.thread_title) # Construct a new nlp doc, which automatically tokenizes and tags, returns a spaCy Doc object 
#     # Go through each token in the post title, only keeping the ones that don't match our stripping rules
#     keep_tokens = [token for token in content_doc if not( # Iterating over a spaCy Doc returns the tokens within the Doc 
#         token.is_stop or # remove stop words, or words that arn't useful for most NLP problems
#         (token.pos_ in ['PROPN','NUM','SYM','X','PUNCT'] and not token.text==".") or # remove proper nouns, numbers, symbols, unknowns, and all punctuation except periods, so we can delineate sentences.
#         #token.is_oov or # Remove words that are out of vocabulary (oov). We might lose some useful data, however, to be safe we take it out as we can't identify it.
#         token.is_space # Remove blank sentences
#     )]
#     print(post.thread_title,keep_tokens) # will print once for each post, so there will be duplicates

In [38]:
import csv
match_docs = []
with open('search_phrases.csv') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        match_docs.append(nlp(row[0]))
match_docs.append(nlp("I'm retarded"))
for doc in match_docs:
    for token in doc:
        print(token, token.pos_)

pulls NOUN
are VERB
the DET
best ADJ
apples NOUN
are VERB
the DET
best ADJ
filthy ADJ
casual NOUN
corn NOUN
cob NOUN
oh INTJ
that DET
’s VERB
funny ADJ
I PRON
'm VERB
retarded ADJ


In [58]:
from spacy.matcher import PhraseMatcher

def on_match(matcher, doc, id, matches):
    pass
    #print('Matched!', matches, doc.text, id)

matcher = PhraseMatcher(nlp.vocab,attr='LOWER')
    
for match_doc in match_docs:
    matcher.add("REEEEE", None, match_doc)
matches = []
for post_tuple in post_tuples:
    if(len(matcher(post_tuple[2]))):
        matches.append(post_tuple)

for match in matches:
    print(match)



(ObjectId('5cc2605bcddd5b478f824c6e'), 9, So everyone who knows what 's what knows that apples or as us enlightened call them pulls are the best . All other fruits and those yucky vegetables are inferior . Discuss .)
(ObjectId('5cc2605bcddd5b478f824c6f'), 14, pulls are the best . The best ' pulls are ripe juicy and taste sooooo good . Nothing else can match I 'd eat apples every day all day if I could doctors be damned)
(ObjectId('5cc2605bcddd5b478f824c71'), 3, Everyone knows grapes are the best you filthy casual Enjoy your cyanide balls)
(ObjectId('5cc2605bcddd5b478f824c79'), 11, Corn cob rows Take me home to the placeeee I belonggg TAKE ME ROWWWSS)
(False, False, i think that i'm retarded)


In [108]:
doc = nlp(raw)
keep_tokens = [token.text for token in doc if not( # Iterating over a spaCy Doc returns the tokens within the Doc 
        token.is_stop or # remove stop words, or words that arn't useful for most NLP problems
        (token.pos_ in ['PROPN','NUM','SYM','X','PUNCT'] and not token.text==".") or # remove proper nouns, numbers, symbols, unknowns, and all punctuation except periods, so we can delineate sentences.
        #token.is_oov or # Remove words that are out of vocabulary (oov). We might lose some useful data, however, to be safe we take it out as we can't identify it.
        token.is_space # Remove blank sentences
    )]
#    print(post.thread_title,keep_tokens) # will print once for each post, so there will be duplicates


# doc_strip = nlp_lg(' '.join(keep_tokens))




In [110]:
for token in doc_strip:
   print('{}, {}, {}'.format(token.text, token.lemma_, token.norm_, token.pos_))
print([(ent.text, ent.label_) for ent in doc_strip.ents])

apple, apple, apple
bananas, banana, bananas
cars, car, cars
turtles, turtle, turtles
carrot, carrot, carrot
[('apple', 'FRUIT')]


In [40]:
doc = doc_strip
from spacy.matcher import Matcher
matcher = Matcher(nlp_lg.vocab)

noun_is_adj_plus = [{"POS": "NOUN"}, {"POS": "ADV", "OP": "*"}, {"POS": "DET", "OP": "*"},
           {"POS": "ADJ", "OP": "+"}]\

noun_is_adj = [{"POS": "NOUN"}, {"POS": "ADJ", "OP": "*"}, {"POS": "ADJ", "OP": "+"}]
noun_is_adv = [{"POS": "NOUN"}, {"POS": "ADV", "OP": "+"}]

adj_noun = [ {"POS": "ADJ"},{"POS": "NOUN"}]

matcher.add("NounIsAdj", None, noun_is_adj)  # add pattern
matcher.add("NounIsAdv", None, noun_is_adv)  # add pattern
matcher.add("AdjNoun", None, adj_noun)

matches = matcher(doc)

fruit_words = []
veggie_words = []

for match_id, start, end in matches:
    string_id = nlp_lg.vocab.strings[match_id]  # Get string representation
    span = doc[start:end]  # The matched span
    #if not set([token.lemma_ for token in span]).isdisjoint(fruit_match_words):
    #    print(span.text)
    
        
    #if set(span.text.split(' ')) & fruit_match_words:
    #    print(span.text)

    print(match_id, string_id, start, end, span.text)

9076452796594085796 NounIsAdv 0 2 Apples best
16820485401210444574 NounIsAdj 15 17 fruits yucky
6136520347023808161 AdjNoun 16 18 yucky vegetables
16820485401210444574 NounIsAdj 17 19 vegetables inferior
6136520347023808161 AdjNoun 27 29 juicy taste
9076452796594085796 NounIsAdv 28 30 taste sooooo
16820485401210444574 NounIsAdj 45 47 Apples gross
6136520347023808161 AdjNoun 49 51 worst fruits
16820485401210444574 NounIsAdj 50 52 fruits bad
6136520347023808161 AdjNoun 66 68 filthy casual
6136520347023808161 AdjNoun 88 90 year!. storms
9076452796594085796 NounIsAdv 98 100 tomatoes Never
16820485401210444574 NounIsAdj 109 111 Peaches best
6136520347023808161 AdjNoun 110 112 best cabbage
16820485401210444574 NounIsAdj 111 113 cabbage worst
6136520347023808161 AdjNoun 112 114 worst end
6136520347023808161 AdjNoun 149 151 Acidic tomatoes
16820485401210444574 NounIsAdj 150 152 tomatoes dirty
6136520347023808161 AdjNoun 151 153 dirty squash
6136520347023808161 AdjNoun 154 156 dirty tomatoes
61

In [24]:
doc = nlp_sm(raw)
from collections import defaultdict
word_count = defaultdict(int)
for token in doc:
    if token.lemma_ in fruit_match_words.union(vegetable_match_words):
        word_count[token.lemma_] += 1

fruit_count = sum([word_count[key] for key in word_count.keys() if key in fruit_match_words])
veg_count = sum([word_count[key] for key in word_count.keys() if key in vegetable_match_words])

print("Fruit mentions: {}".format(fruit_count))
print("Veg mentions: {}".format(veg_count))

Fruit mentions: 20
Veg mentions: 15


In [83]:
#posts = collection.find({})

noun_adj_pairs = []

for sent in doc.sents:
    for i,token in enumerate(sent):
        if token.pos_ not in ('NOUN','PROPN', 'ADJ'):
            continue
        for j in range(i+1,len(sent)):
            if (token.pos_ in ('NOUN','PROPN') and sent[j].pos_ == 'ADJ'):
                noun_adj_pairs.append((token,sent[j]))
                break
            if (token.pos_ == 'ADJ' and sent[j].pos_ in ('NOUN','PROPN')):
           #    noun_adj_pairs.append((sent[j], token))
                break
    
fruit_pairs = [(pair[0].lemma_,pair[1].lemma_) for pair in noun_adj_pairs if pair[0].lemma_ in fruit_match_words]
veg_pairs = [(pair[0].lemma_,pair[1].lemma_) for pair in noun_adj_pairs if pair[0].lemma_ in vegetable_match_words]

print(fruit_pairs)


[('apple', 'good'), ('fruit', 'yucky'), ('apple', 'gross'), ('fruit', 'which'), ('grape', 'superior'), ('peach', 'good'), ('peach', 'good')]


In [ ]:
from spacy.lang.en import English
from spacy.pipeline import EntityRuler

ruler = EntityRuler(nlp_lg)
patterns = [{"label": "FRUIT", "pattern": [{"lower":"apple"}]},
            {"label": "FRUIT", "pattern": [{"lower":"grape"}]},
           ]
ruler.add_patterns(patterns)
nlp.add_pipe(ruler)

threads = collection.find({"title":"Apples ('pulls) are the best fruit"})
threads = [thread['posts'] for thread in threads]
posts = [post for thread in threads for post in thread]
contents = [post['content'] for post in posts]
pprint(contents)

